**NOTE** `phimats` environment should be used as kernel

In [1]:
import h5py

import PreProcessing
from PreProcessing import PreProcessing as PP, ReadMesh

import BoundaryConditions
from BoundaryConditions import *

import PostProcessing
from PostProcessing import WriteXDMF

### Simulation data

In [2]:
# Simulation name
Simul = "HLGB"
# Simulation type
SimulType = "HLGBTrapping"
# Element name
elementName = "triangle"  
# Element sets
nElementSets = 1
# Number of steps to apply the load
nSteps = 100

### Read mesh file 

The mesh file is hosted on Zenodo due to its large size.  
To download it, paste the command below into your terminal.  
**Note:** Make sure you're in the desired working directory.

```bash
wget https://zenodo.org/records/15380594/files/RVE_mesh.inp
```

In [3]:
modelName = "RVE_mesh"

mesh = ReadMesh(modelName)
print(mesh)

<meshio mesh object>
  Number of points: 1000000
  Number of cells:
    triangle: 1996002
  Point sets: plate
  Cell sets: plate


### Element data

In [4]:
elementName = "triangle" 	# meshio compatible element name
nNodes = mesh.points.shape[0]            # Total number of nodes
nodCoord = mesh.points[:,0:2]            # Node coordinates
nElements = mesh.cells_dict[elementName].data.shape[0]  # Total number of elements
elemNodeConn = mesh.cells_dict[elementName]        # Node connectivity for all elements

### Read phase-field data $g(\phi)$

In [5]:
# gPhi mapped to finite element mesh

fh5 = h5py.File("RVEMori.hdf5", "r")

try:
	gPhi_HAGB = fh5["gPhi_HAGB"][()]    
	gPhi_LAGB = fh5["gPhi_LAGB"][()]    
	fh5.close()
except:
    fh5.close()


## Diffusion and trapping data

In [6]:
# Diffusivity data
T = 300                    # Temperature [K]
R = 8.31446261815324       # Universal gas constant [J/mol.K]
dt = 10e-3  			   # Time increment [s]
N = 6          	# Number of interstitial sites per reference lattice atom [tetrahedral sites in BCC]

D01 = 8.45e-8; DQ1 = 5000  # Diffusivity m²/s
DL = D01*np.exp(-DQ1/(R*T))

H_HAGB = 15000 # HAGB enrichment enthalpy[J/mol]
H_LAGB = 10000 # LAGB enrichment enthalpy[J/mol]

Vm = 7.09e-6  # Molar volume of Fe [m³/mol]
Vgb = 2*Vm    # Molar volume around GBs  [m³/mol]

kappa_HAGB = 4*N*H_HAGB/Vm # HAGBGB occupancy enrichment ratio
KHAGB = np.exp(0.25*kappa_HAGB*Vm/(N*R*T))
print("KHAGB: ", KHAGB)

kappa_LAGB = 4*N*H_LAGB/Vm # LAGB occupancy enrichment ratio
KLAGB = np.exp(0.25*kappa_LAGB*Vm/(N*R*T))
print("KLAGB: ", KLAGB)

theta_b = 3.453e-3*Vm/N
print(theta_b)

cL = theta_b*N/Vm  # Concentration  [mol/m³]
print("cL", cL)

theta_HAGB = theta_b*KHAGB # Occupancy at the HAGB
theta_LAGB = theta_b*KLAGB # Occupancy at the LAGB

c_HAGB = theta_HAGB/Vgb
print("c_HAGB", c_HAGB)
Z_HAGB = c_HAGB/cL # Concentration enrichment ratio
print("Z_HAGB", Z_HAGB)
zeta_HAGB = 4*R*T*np.log(Z_HAGB)
print("zeta_HAGB", zeta_HAGB)

c_LAGB = theta_LAGB/Vgb
print("c_LAGB", c_LAGB)
Z_LAGB = c_LAGB/cL # Concentration enrichment ratio
print("Z_LAGB", Z_LAGB)
zeta_LAGB = 4*R*T*np.log(Z_LAGB)
print("zeta_LAGB", zeta_LAGB)

print("Eq Con HAGB: ", np.exp(0.25*zeta_HAGB/(R*T)))
print("Eq Con LAGB: ", np.exp(0.33*zeta_LAGB/(R*T)))

KHAGB:  408.9601637423769
KLAGB:  55.09607620783739
4.080295e-09
cL 0.003453
c_HAGB 0.11767828711686895
Z_HAGB 34.08001364519807
zeta_HAGB 35207.20386088472
c_LAGB 0.01585389592880521
Z_LAGB 4.591339683986449
zeta_LAGB 15207.20386088472
Eq Con HAGB:  34.08001364519806
Eq Con LAGB:  7.477562307558887


In [7]:
# Material parameters dict

Materials = {
    "Material_1" : {
	"D0x1" : D01,
 	"D0y1" : D01,
	"DQx1" : DQ1,
	"DQy1" : DQ1,
 	"zeta_HAGB"  : zeta_HAGB,
  	"zeta_LAGB"  : zeta_LAGB,
	}
}

Materials

{'Material_1': {'D0x1': 8.45e-08,
  'D0y1': 8.45e-08,
  'DQx1': 5000,
  'DQy1': 5000,
  'zeta_HAGB': np.float64(35207.20386088472),
  'zeta_LAGB': np.float64(15207.20386088472)}}

### Apply boundary conditions

In [8]:
lx = 100e-6    # width of the RVE
ly = 100e-6    # height of the RVE

exitNods = []

# Set keyworkd argument `Charging` to true for charging simulation
conBCs, exitNods = PermeationX(ly, cL, mesh, Charging=False)

# Equilibrium boundary conditions
for i in range(len(conBCs)):
    conBCs[i][1] = conBCs[i][1]*np.exp((gPhi_HAGB[conBCs[i][0]]*zeta_HAGB+gPhi_LAGB[conBCs[i][0]]*zeta_LAGB)/(R*T))
    
# Write boundary conditions in a .vtu file for checking in Paraview. recommended view is `Points`. 
BCmesh = WriteConBCs(Simul, elementName, mesh, conBCs)

### Initialize simulation object

In [10]:
SimulationData = {
    "Simul"            : Simul,
    "SimulType"		   : SimulType,
    "mesh"             : mesh,
    "gPhi_HAGB"        : gPhi_HAGB,
    "gPhi_LAGB"        : gPhi_LAGB,
    "elementName"      : elementName,
    "nElementSets"     : nElementSets,
    "presBCs"          : conBCs,
    "conB"             : cL,
    "exitNods"         : exitNods,
    "dt"               : dt,
    "nSteps"           : nSteps,
    "Materials"        : Materials,
    "T"                : T,
}

# Preprocessing object
GB_Sim = PP(SimulationData)

# Create the input file Simul+"_in.hdf5" file
GB_Sim.WriteInputFile()

HDF5 file 'HLGB_in.hdf5' opened successfully in write mode.
HDF5 file closed successfully.


In [11]:
# Create the output file Simul+"_out.hdf5" file
GB_Sim.WriteOutputFile(OVERWRITE=True, FLUX=True, AVFLUX=True)

### Generate xdmf visualization file

In [12]:
WriteXDMF(Simul, elementName, nSteps+1, "Transport2D", FLUX=True)